In [6]:
import os
import zipfile
import pandas as pd
from datetime import datetime
from app.models.engine.app import db
from app.models import ElectionDate, ElectionRace, ElectionRaceCandidate, State, County, Precinct

DOWNLOAD_DIR = os.path.join(
    os.getcwd(), 'election-results-scraper/scraped_files')
SOURCE_DIR = os.path.join(os.getcwd(), 'data/source/')
COUNTY_RESULTS_DIR = os.path.join(SOURCE_DIR, 'county_election_results')
PRECINCT_RESULTS_DIR = os.path.join(SOURCE_DIR, 'precinct_election_results')
STATE_RESULTS_DIR = os.path.join(SOURCE_DIR, 'state_election_results')


def unzipCountyResults():
    countyZipFiles = [f for f in os.listdir(DOWNLOAD_DIR) if f.split('_')[
        1].startswith('County')]
    currentSourceFiles = [f for f in os.listdir(COUNTY_RESULTS_DIR)]
    countyZipFiles = [
        z for z in countyZipFiles if z.replace('_csv.zip', '.csv') not in currentSourceFiles]
    print(f'Unzipping {len(countyZipFiles)} county results files...')
    for f in countyZipFiles:
        with zipfile.ZipFile(os.path.join(DOWNLOAD_DIR, f), 'r') as z:
            z.extractall(COUNTY_RESULTS_DIR)


def unzipPrecinctResults():
    precinctZipFiles = [f for f in os.listdir(DOWNLOAD_DIR) if f.split('_')[
        1].startswith('Precinct')]
    currentSourceFiles = [f for f in os.listdir(PRECINCT_RESULTS_DIR)]
    precinctZipFiles = [
        z for z in precinctZipFiles if z.replace('_csv.zip', '.csv') not in currentSourceFiles]
    print(f'Unzipping {len(precinctZipFiles)} precinct results files...')
    for f in precinctZipFiles:
        with zipfile.ZipFile(os.path.join(DOWNLOAD_DIR, f), 'r') as z:
            z.extractall(PRECINCT_RESULTS_DIR)


def getCountyResultsDF():
    unzipCountyResults()
    source_files = [f for f in os.listdir(COUNTY_RESULTS_DIR)]
    df = pd.concat([pd.read_csv(os.path.join(COUNTY_RESULTS_DIR, f),
                                engine='python', delimiter=",", encoding='utf-8', index_col=False) for f in source_files])
    df['elec_date'] = df.apply(
        lambda r: datetime.strptime(r['elec_date'], '%m/%d/%Y'), axis=1)
    df['elec_date_id'] = df.apply(
        lambda r: int(r['elec_date'].strftime('%Y%m%d')), axis=1)
    return df


def getPrecinctResultsDF():
    unzipPrecinctResults()
    source_files = [f for f in os.listdir(PRECINCT_RESULTS_DIR)]
    df = pd.concat([pd.read_csv(os.path.join(PRECINCT_RESULTS_DIR, f),
                                engine='python', delimiter=",", encoding='utf-8', index_col=False) for f in source_files])
    df['elec_date'] = df.apply(
        lambda r: datetime.strptime(r['elec_date'], '%m/%d/%Y'), axis=1)
    df['elec_date_id'] = df.apply(
        lambda r: int(r['elec_date'].strftime('%Y%m%d')), axis=1)
    return df


In [31]:
df_counties = pd.read_csv(os.path.join(SOURCE_DIR, 'counties/ok_counties.csv'))
County.load_dataframe(df_counties.drop(
    columns=['geoid']), column_mappings={"geoid": 'id'})
df_counties.head()


,Unnamed: 0,id,name,geoid,state_id,SHAPE_Area,SHAPE_center_lat,SHAPE_center_lon,total_precincts,geometry
0,0,109,Oklahoma,40109,40,1.865225e+09,35.551913,-97.407092,257,"POLYGON((-97.6713047525515 35.4063168600549, -..."
1,1,25,Cimarron,40025,40,4.769986e+09,36.748208,-102.517417,6,"POLYGON((-103.002541624433 36.90923463726, -10..."
2,2,151,Woods,40151,40,3.337247e+09,36.766679,-98.865521,6,"POLYGON((-99.129133860706 36.99896107172, -99...."
3,3,59,Harper,40059,40,2.688834e+09,36.788294,-99.667949,6,"POLYGON((-100.003831325599 36.5926997156928, -..."
4,4,143,Tulsa,40143,40,1.516677e+09,36.119505,-95.939560,262,"POLYGON((-96.2965277269222 36.0767727175477, -..."


In [33]:
df_precincts = pd.read_csv(os.path.join(SOURCE_DIR, 'precincts/ok_precincts.csv'))
df_precincts.head()

,Unnamed: 0,id,ok_district_id,precinct_num,county_id,county_number,ok_commissioner_district_id,ok_house_district_id,ok_senate_district_id,us_congressional_district_id,SHAPE_Area,SHAPE_center_lat,SHAPE_center_lon
0,0,500009,9,9,99,50,2,22,14,4,8.661510e+07,34.590022,-96.986370
1,1,500015,15,15,99,50,1,22,14,4,1.891817e+08,34.428643,-96.935578
2,2,430204,4,204,85,43,2,49,14,4,2.933666e+07,33.959306,-97.166968
3,3,430101,1,101,85,43,1,49,14,4,1.055144e+08,33.891767,-97.067053
4,4,270030,7,30,53,27,1,38,19,3,4.079621e+08,36.908931,-97.770127


In [8]:
df_county_results = getCountyResultsDF()
df_county_results.head()

Unzipping 0 county results files...


,elec_date,county,entity_description,race_number,race_description,race_party,tot_race_prec,race_prec_reporting,cand_number,cand_name,cand_party,cand_absmail_votes,cand_early_votes,cand_elecday_votes,cand_tot_votes,race_county_owner,elec_date_id
0,2017-04-04,ALFALFA,CITY OF CHEROKEE,40201,FOR COMMISSIONER WARD 1 CITY OF CHEROKEE,NaN,1,1,1,MICHAEL LEE WEST,NaN,0,0,8,8,ALFALFA,20170404
1,2017-04-04,ALFALFA,CITY OF CHEROKEE,40201,FOR COMMISSIONER WARD 1 CITY OF CHEROKEE,NaN,1,1,2,LANCE MILLER,NaN,0,1,29,30,ALFALFA,20170404
2,2017-04-04,ALFALFA,CITY OF CHEROKEE,40204,PROPOSITION CITY OF CHEROKEE,NaN,2,2,1,FOR THE PROPOSITION - YES,NaN,2,8,63,73,NaN,20170404
3,2017-04-04,ALFALFA,CITY OF CHEROKEE,40204,PROPOSITION CITY OF CHEROKEE,NaN,2,2,2,AGAINST THE PROPOSITION - NO,NaN,1,0,7,8,NaN,20170404
4,2017-04-04,ALFALFA,TOWN OF CARMEN,40205,FOR BOARD OF TRUSTEES TOWN OF CARMEN,NaN,1,1,1,TAMMY JEAN PRUITT,NaN,1,0,20,21,ALFALFA,20170404


In [9]:

# import TEMP_df_county_results
connection = db.engine.raw_connection()
cursor = connection.cursor()
command = "DROP TABLE IF EXISTS TEMP_df_county_results;"
cursor.execute(command)
connection.commit()
cursor.close()

df_county_results.to_sql('TEMP_df_county_results',
                         con=db.engine, if_exists='replace')


In [29]:
# Load missing county-level results to election_results
q = """
    SELECT 
        t.race_number, 
        t.elec_date,
        t.elec_date_id,
        t.entity_description as race_entity_description,
        t.race_county_owner,
        t.race_description,
        t.race_party,
        t.tot_race_prec as race_tot_prec,	
        t.race_prec_reporting,
        SUM(t.cand_tot_votes) as race_tot_votes,
        SUM(t.cand_absmail_votes) as race_absmail_votes,
        SUM(t.cand_early_votes) as race_early_votes,	
        SUM(t.cand_elecday_votes) as race_elecday_votes,
        MAX(t.cand_number) as race_num_candidates
    FROM TEMP_df_county_results as t
    LEFT JOIN election_races as r
        on t.elec_date_id = r.elec_date_id
        and t.race_number = r.race_number
        and t.race_description like r.race_description
    WHERE r.id is null
    GROUP BY t.race_number, 
        t.elec_date,
        t.elec_date_id,
        t.entity_description,
        t.race_county_owner,
        t.race_description,
        t.race_party,
        t.tot_race_prec,	
        t.race_prec_reporting
    
     """
df_missing_election_races = pd.read_sql(q, con=db.engine)
ElectionRace.load_dataframe(df_missing_election_races)


ElectionRace.load_dataframe: 49 rows
  Missing Table columns: id


{'rows': 8433}

In [30]:
df_county_results.head()

,elec_date,county,entity_description,race_number,race_description,race_party,tot_race_prec,race_prec_reporting,cand_number,cand_name,cand_party,cand_absmail_votes,cand_early_votes,cand_elecday_votes,cand_tot_votes,race_county_owner,elec_date_id
0,2017-04-04,ALFALFA,CITY OF CHEROKEE,40201,FOR COMMISSIONER WARD 1 CITY OF CHEROKEE,NaN,1,1,1,MICHAEL LEE WEST,NaN,0,0,8,8,ALFALFA,20170404
1,2017-04-04,ALFALFA,CITY OF CHEROKEE,40201,FOR COMMISSIONER WARD 1 CITY OF CHEROKEE,NaN,1,1,2,LANCE MILLER,NaN,0,1,29,30,ALFALFA,20170404
2,2017-04-04,ALFALFA,CITY OF CHEROKEE,40204,PROPOSITION CITY OF CHEROKEE,NaN,2,2,1,FOR THE PROPOSITION - YES,NaN,2,8,63,73,NaN,20170404
3,2017-04-04,ALFALFA,CITY OF CHEROKEE,40204,PROPOSITION CITY OF CHEROKEE,NaN,2,2,2,AGAINST THE PROPOSITION - NO,NaN,1,0,7,8,NaN,20170404
4,2017-04-04,ALFALFA,TOWN OF CARMEN,40205,FOR BOARD OF TRUSTEES TOWN OF CARMEN,NaN,1,1,1,TAMMY JEAN PRUITT,NaN,1,0,20,21,ALFALFA,20170404


In [ ]:
# Load missing county-level results to election_results
q = """
    SELECT 
        t.race_number, 
        t.elec_date,
        t.elec_date_id,
        t.entity_description as race_entity_description,
        t.race_county_owner,
        t.race_description,
        t.race_party,
        t.tot_race_prec as race_tot_prec,	
        t.race_prec_reporting,
        SUM(t.cand_tot_votes) as race_tot_votes,
        SUM(t.cand_absmail_votes) as race_absmail_votes,
        SUM(t.cand_early_votes) as race_early_votes,	
        SUM(t.cand_elecday_votes) as race_elecday_votes,
        MAX(t.cand_number) as race_num_candidates
    FROM TEMP_df_county_results as t
    INNER JOIN election_races as r
        on t.elec_date_id = r.elec_date_id
        and t.race_number = r.race_number
        and t.race_description like r.race_description
    WHERE r.id is null
    GROUP BY t.race_number, 
        t.elec_date,
        t.elec_date_id,
        t.entity_description,
        t.race_county_owner,
        t.race_description,
        t.race_party,
        t.tot_race_prec,	
        t.race_prec_reporting
    
     """